In [45]:
# Imports

import gspread as gs
import pandas as pd
import plotly.graph_objects as go

In [46]:
# Match Results Google Sheet
sheet_id = "1RBWOGn_cBeK9qQmSbVz7NE_XGTBwevMVOFWbW9EHGf8"
sheet_name = "Match_Results_2"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
results_df = pd.read_csv(url).fillna('')
results_df = results_df.sort_values('Match Week', ascending=False)
# results_df.head(50)

In [47]:
# Creating Won_Loss Results Table
winners = dict(enumerate(results_df['Match Winner']))
losers = dict(enumerate(results_df['Match Loser']))
winners.update(losers)
players = winners
wins = results_df['player1'].value_counts()
losses = results_df['player2'].value_counts()
w_l = pd.concat([wins,losses],axis=1).fillna(0).astype(int).round().rename(columns={'player1':'Wins','player2':'Losses'})
w_l.loc[:,'Matches']=w_l.sum(numeric_only=True, axis=1)
w_l['Win%'] = w_l['Wins']/w_l['Matches']
w_l['Player']=w_l.index
column_names = ['Player',"Matches", "Wins", "Losses", "Win%"]
w_l['Win%'] = w_l['Win%'].round(3)
w_l = w_l.reindex(columns=column_names)
w_l = w_l.reset_index(drop=True)
# w_l.head(10)

In [48]:
# Rankings
rankings_df = pd.read_csv('Rankings.csv')
rankings_df.drop_duplicates(subset=['Player'],inplace=True)
rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.loc[:,'Elo']=1600
initial_ranking = 1600
k_factor = 32
# rankings_df.head(20)

In [49]:
# Elo Formula
def elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor):
    R1 = 10**(p1_ranking/400)
    R2 = 10**(p2_ranking/400)
    exp_1 = R1/(R1+R2)
    exp_2 = R2/(R1+R2)
    Elo_1 = p1_ranking+k_factor*(p1_score-exp_1)
    Elo_2 = p2_ranking+k_factor*(p2_score-exp_2)
    return Elo_1, Elo_2

In [50]:
#this loop reads the scores in order starting from the top of the matches
#every time you run this loop it will read all of the scores again
for idx in results_df.index:
    player_1 = results_df.player1[idx]
    p1_score = results_df.player1score[idx]
    player_2 = results_df.player2[idx]
    p2_score = results_df.player2score[idx]
    current_rankings_list = list(set(rankings_df.index))
    if player_1 in current_rankings_list:
        p1_ranking = rankings_df.at[player_1,'Elo']
    else:
        p1_ranking = initial_ranking
    if player_2 in current_rankings_list:
        p2_ranking = rankings_df.at[player_2,'Elo']
    else:
        p2_ranking = initial_ranking
    
    Elo_1, Elo_2 = elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor)
    
    if player_1 in current_rankings_list:
        rankings_df.at[player_1,'Elo'] = Elo_1
    else:
        new_Elo_dict = {'Player':player_1,'Elo':Elo_1}
        rankings_df = rankings_df.append(new_Elo_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
    if player_2 in current_rankings_list:
        rankings_df.at[player_2,'Elo'] = Elo_2
    else:
        new_Elo_dict = {'Player':player_2,'Elo':Elo_2}
        rankings_df = rankings_df.append(new_Elo_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.reset_index(drop=True,inplace=True)
rankings_df.sort_values(by=['Elo'],inplace=True,ascending=False,ignore_index=True)
rankings_df.index+=1
rankings_df.to_csv('Rankings.csv',index=False)

In [51]:
pd.set_option('display.max_rows', None)
rankings_df

,Player,ELO,Elo
1,Mike Dunlap,1684.0,1718
2,Missy Kayko,1670.0,1695
3,Ben Brown,1678.0,1682
4,Jeff Ziev,1630.0,1679
5,Bob Sauchelli,1617.0,1652
6,Eric Papa,1631.0,1650
7,Mike Brady,1600.0,1650
8,Nelson Bakerman,1640.0,1649
9,Paul Assad,1617.0,1648
10,Ryan Leggette,1627.0,1648


In [52]:
# Formatting and Printing Final Table
final_table = w_l.merge(rankings_df, how="left", on="Player")
final_table_print = final_table.drop('Matches', axis=1)
final_table_print['Played'] = final_table_print['Wins'] + final_table_print['Losses']
final_table_print = final_table_print[final_table_print['Played'] >= 5]
final_table_print['Ranking'] = final_table_print[["Elo","Win%"]].apply(tuple,axis=1)\
             .rank(ascending=False, method="min").astype(int)
final_table_print['Ranking'] = final_table_print['Ranking'].round(3)
final_table_print['Elo'] = final_table_print['Elo'].round(0)
column_names = ['Ranking','Player', "Elo", "Wins", "Losses", "Win%"]
final_table_print = final_table_print.reindex(columns=column_names)
final_table_print = final_table_print.sort_values(by=['Ranking','Win%'],ascending=[True,False])
final_table_print

,Ranking,Player,Elo,Wins,Losses,Win%
0,1,Mike Dunlap,1718,11,2,0.846
1,2,Missy Kayko,1695,10,3,0.769
8,3,Ben Brown,1682,5,0,1.000
2,4,Jeff Ziev,1679,10,3,0.769
7,5,Bob Sauchelli,1652,6,2,0.750
9,6,Mike Brady,1650,5,1,0.833
5,7,Eric Papa,1650,7,3,0.700
4,8,Nelson Bakerman,1649,7,3,0.700
6,9,Ryan Leggette,1648,6,2,0.750
3,10,Paul Assad,1648,7,3,0.700


In [53]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
    header=dict(values=list(column_names),
                fill_color='darkblue',
                font_color='white',
                font_size=16,
                align='center'),
    cells=dict(values=[
                final_table_print.Ranking,
                final_table_print.Player,
                final_table_print.Elo,
                final_table_print.Wins,
                final_table_print.Losses, 
                final_table_print['Win%']
                ],
               fill_color='white',
               font_color='black',
               font_size=14,
               align='center',
               height=30,
               line_color='lightgray',
               format=["1234","","#","#","#",".1%"]
               ),
    columnwidth=20)])

fig.update_layout(
    autosize=False,
    height=2400,
    width=1100)
fig.show()
fig.write_html("final_table.html", include_plotlyjs="cdn")

# final_table_print.to_csv('Rankings_Table.csv',index=False)
# final_table_print.to_excel('Rankings_Table.xlsx',index=False)